In [1]:
import pandas as pd
import altair as alt

In [2]:
df = pd.read_csv("../data/VEH0105/regions.csv")

In [3]:
df = df[(df['BodyType']!="Total") & (df['Keepership']!="Total") & (df['Fuel']!="Total")]

In [4]:
df[[col for col in df.columns[6:]]] = df[[col for col in df.columns[6:]]].apply(pd.to_numeric, errors="coerce")

In [16]:
bt_fuel = df.groupby(["BodyType", "Fuel"])[["2022 Q3"]].sum().reset_index()

In [17]:
bt_fuel['2022 Q3'] = bt_fuel['2022 Q3'] / 1000

In [40]:
order = ['Diesel', 'Petrol', 'Other fuels']
categories = ['Motorcycles', 'Cars', 'Light goods vehicles', 'Heavy goods vehicles', 'Buses and coaches', 'Other vehicles']
#colors = ['#f03e3e', '#1098ad', '#37b24d']

In [308]:
chart = alt.Chart(data=bt_fuel, height=300, width=600, title="UK vehicles by category and fuel type").mark_bar().encode(

alt.Y('BodyType:N', sort=categories, title=None),
alt.X('2022 Q3:Q', title="Vehicles (millions)"),
alt.Color('Fuel:N', sort=order), #scale=alt.Scale(domain=order, range=colors)),
alt.Order("color_Fuel_sort_index:Q"),
#alt.Tooltip(field='2022 Q3', format=",.1f")
alt.Tooltip(['Fuel', '2022 Q3'])#, format=",.1f")
).configure_title(anchor="middle", fontSize=16, dy=-16)

In [309]:
chart

alt.Chart(...)

In [310]:
chart.save('../charts/VEH0105_veh_cat_fuel.json')

In [154]:
df.groupby(['BodyType', 'Keepership'])[['2022 Q3']].sum().reset_index()

,BodyType,Keepership,2022 Q3
0,Buses and coaches,Company,114.767
1,Buses and coaches,Private,29.031
2,Cars,Company,2779.488
3,Cars,Private,29739.392
4,Heavy goods vehicles,Company,427.741
5,Heavy goods vehicles,Private,104.436
6,Light goods vehicles,Company,2187.075
7,Light goods vehicles,Private,2372.133
8,Motorcycles,Company,39.965
9,Motorcycles,Private,1399.925


In [155]:
cars = df[df['BodyType'] == 'Cars']

In [156]:
keep_fuel = cars.groupby(['Keepership', 'Fuel'])[['2022 Q3']].sum().reset_index()
keep_fuel['2022 Q3'] = keep_fuel['2022 Q3'] / 1000

In [157]:
keep_fuel

,Keepership,Fuel,2022 Q3
0,Company,Diesel,0.755738
1,Company,Other fuels,0.805266
2,Company,Petrol,1.218484
3,Private,Diesel,10.832666
4,Private,Other fuels,1.323874
5,Private,Petrol,17.582852


In [227]:
chart2 = alt.Chart(data=keep_fuel, height=150, width=600, title="UK cars by keepership and fuel type").mark_bar().encode(

alt.Y('Fuel:N', title=None, sort=['Diesel', 'Petrol', 'Other fuels']),
alt.X('2022 Q3:Q', title="Cars (millions)"),
alt.Color('Fuel:N', sort=['Diesel', 'Petrol', 'Other fuels']), #scale=alt.Scale(domain=order, range=colors)),
alt.Row('Keepership:N', title=None),
alt.Tooltip(['Fuel', '2022 Q3'])
#alt.TitleConfig(align="left")
).configure_title(anchor="middle", fontSize=16, dy=-16)#.configure(font="Roboto")


In [228]:
chart2.save('../charts/VEH0105_car_keep_fuel.json')

In [229]:
chart2

alt.Chart(...)

In [237]:
alt_cars = cars[cars['Fuel']=='Other fuels']

In [238]:
geo_alt = alt_cars.groupby(['ONS Geography'])[['2022 Q3']].sum().reset_index()

In [239]:
geo_alt

,ONS Geography,2022 Q3
0,East,206.910
1,East Midlands,124.629
2,London,287.317
3,North East,47.558
4,North West,249.304
5,Northern Ireland,38.684
6,Scotland,139.169
7,South East,425.771
8,South West,232.775
9,Wales,59.232


In [240]:
geo_all = cars.groupby(['ONS Geography'])[['2022 Q3']].sum().reset_index()

In [241]:
geo_all

,ONS Geography,2022 Q3
0,East,3406.949
1,East Midlands,2495.668
2,London,2610.116
3,North East,1181.064
4,North West,3563.107
5,Northern Ireland,1002.817
6,Scotland,2538.411
7,South East,5321.818
8,South West,3298.428
9,Wales,1601.654


In [299]:
geo = geo_all.copy()

In [300]:
geo['Other fuels'] = geo_alt['2022 Q3']

In [301]:
geo['Other fuels %'] = (geo['Other fuels'] / geo['2022 Q3'])*100

In [302]:
geo.rename(columns={'ONS Geography': 'Region', '2022 Q3': 'All cars'}, inplace=True)

In [303]:
geo = geo.sort_values(by="Other fuels %")

In [304]:
geo[['All cars', 'Other fuels']] = (geo[['All cars', 'Other fuels']] * 1000).astype(int)

In [305]:
geo

,Region,All cars,Other fuels,Other fuels %
9,Wales,1601654,59232,3.698177
5,Northern Ireland,1002817,38684,3.857533
3,North East,1181063,47558,4.026708
10,West Midlands,2892511,143063,4.945979
1,East Midlands,2495667,124629,4.993813
6,Scotland,2538411,139168,5.482524
0,East,3406949,206910,6.073176
11,Yorkshire and The Humber,2606337,174728,6.703968
4,North West,3563107,249304,6.996815
8,South West,3298428,232775,7.057150


In [307]:
geo.to_html("region_alt_fuels_table.html", index=False, border=0, table_id="region-alt-fuels", float_format='{:10.1f}'.format)